In [52]:
from dataclasses import dataclass
import re


@dataclass
class ScratchCard:
    card_id: int

    winning_numbers: list[int]
    dealt_numbers: list[int]

    matches: list[int]
    points: int = 0

    def set_matches(self) -> None:
        self.matches = [
            num for num in self.dealt_numbers if num in self.winning_numbers
        ]

    def get_points(self) -> int:
        self.set_matches()

        n = len(self.matches)
        def double_n_minus_1_times(n: int): return 2 ** (n - 1)

        return 0 if n == 0 else double_n_minus_1_times(n)


def parse_line_into_scratchcard(line: str, line_y: int) -> ScratchCard:
    card_content = re.search(r"(?:Card\s+\d+:)(.*)", line)

    if card_content is None:
        raise ValueError()

    card_content = card_content.group(1)
    winning_numbers, dealt_numbers = card_content.split("|")

    winning_numbers = [int(str_num) for str_num in winning_numbers.split()]
    dealt_numbers = [int(str_num) for str_num in dealt_numbers.split()]

    scratchcard = ScratchCard(
        line_y, winning_numbers, dealt_numbers, matches=[]
    )

    return scratchcard


def get_copies(
    card: ScratchCard, card_map: dict[int, ScratchCard]
) -> list[ScratchCard]:
    card_id, n_matches = card.card_id, len(card.matches)

    if n_matches == 0:
        return [card]

    else:
        all_copies = []
        next_cards = [
            card_map[i] for i in range(card_id + 1, card_id + n_matches + 1)
        ]
        for next_card in next_cards:
            next_card_copies = get_copies(next_card, card_map)
            all_copies.extend(next_card_copies)

        all_copies.append(card)
        return all_copies

In [54]:
def main_part_1():
    with open("input.txt", "r") as file:
        lines = file.readlines()

    scratchcards = [
        parse_line_into_scratchcard(line, y)
        for y, line in enumerate(lines, start=1)
    ]
    points = [card.get_points() for card in scratchcards]

    result = sum(points)
    print(result)


def main_part_2():
    with open("input.txt", "r") as file:
        lines = file.readlines()

    scratchcards = [
        parse_line_into_scratchcard(line, y)
        for y, line in enumerate(lines, start=1)
    ]
    for card in scratchcards:
        card.set_matches()

    card_map = {card.card_id: card for card in scratchcards}

    all_copies = []
    for card in scratchcards:
        card_copies = get_copies(card, card_map)
        all_copies.extend(card_copies)

    result = len(all_copies)
    print(result)


main_part_1()
main_part_2()

33950
14814534


In [53]:
# example validation

from collections import Counter

example_lines = [
    "Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53",
    "Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19",
    "Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1",
    "Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83",
    "Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36",
    "Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11",
]

example_scratchcards = [
    parse_line_into_scratchcard(line, y)
    for y, line in enumerate(example_lines, start=1)
]
for sc in example_scratchcards:
    print(sc)

print()

example_points = [card.get_points() for card in example_scratchcards]
for point in example_points:
    print(point)

print()

example_card_map = {card.card_id: card for card in example_scratchcards}
for key, value in example_card_map.items():
    print(f"{key}: {value}")

print()

example_all_copies = []
for card in example_scratchcards:
    card_copies = get_copies(card, example_card_map)
    example_all_copies.extend(card_copies)

card_ids = [card.card_id for card in example_all_copies]
counts = Counter(card_ids)
for num, count in counts.items():
    print(f"{num}: {count}")

print("Total scratchcards: ", len(example_all_copies))

ScratchCard(card_id=1, winning_numbers=[41, 48, 83, 86, 17], dealt_numbers=[83, 86, 6, 31, 17, 9, 48, 53], matches=[], points=0)
ScratchCard(card_id=2, winning_numbers=[13, 32, 20, 16, 61], dealt_numbers=[61, 30, 68, 82, 17, 32, 24, 19], matches=[], points=0)
ScratchCard(card_id=3, winning_numbers=[1, 21, 53, 59, 44], dealt_numbers=[69, 82, 63, 72, 16, 21, 14, 1], matches=[], points=0)
ScratchCard(card_id=4, winning_numbers=[41, 92, 73, 84, 69], dealt_numbers=[59, 84, 76, 51, 58, 5, 54, 83], matches=[], points=0)
ScratchCard(card_id=5, winning_numbers=[87, 83, 26, 28, 32], dealt_numbers=[88, 30, 70, 12, 93, 22, 82, 36], matches=[], points=0)
ScratchCard(card_id=6, winning_numbers=[31, 18, 13, 56, 72], dealt_numbers=[74, 77, 10, 23, 35, 67, 36, 11], matches=[], points=0)

8
2
2
1
0
0

1: ScratchCard(card_id=1, winning_numbers=[41, 48, 83, 86, 17], dealt_numbers=[83, 86, 6, 31, 17, 9, 48, 53], matches=[83, 86, 17, 48], points=0)
2: ScratchCard(card_id=2, winning_numbers=[13, 32, 20, 16, 